# A hands-on introduction to Natural Language Processing in Python
#### 16th July, 2019 at OSCON
## Pre-requisites
Access the notebook at 

Please have the following pre-requisites ready on your machine. If you don't have them, please install them with the hurry of this hacker cat!

![Typing cat gif](https://media.giphy.com/media/o0vwzuFwCGAFO/giphy.gif)

### Installing Jupyter on your machine

- If you already have Anaconda, you should have Jupyter installed as well. ```jupyter notebook``` opens up a Jupyter notebook in your browser at default port 8888. 

- Use pip install. ```python3 -m pip install jupyter``` for Python 3 install, ```python -m pip install jupyter``` for Python 2 install. For more details, refer to https://jupyter.org/install

- Also use pip to install the following packages: ```nltk, lxml, requests, matplotlib, re, sklearn, wikipedia, random, gensim, wordcloud```

## NLTK
Natural Language Toolkit is the most popular collection of libraries and programs to do NLP. You can find more about it here: http://www.nltk.org/book/ch00.html. We will mainly be using NLTK to perform different tasks, along with a few other packages.

In [2]:
import nltk
nltk.download('punkt')
nltk.download('gutenberg')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/billyfisher/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package gutenberg to
[nltk_data]     /Users/billyfisher/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


True

NLTK comes pre-loaded with texts from the Project Gutenberg archive that you can use. It also has a collection of informal text from discussion forums, conversations, chat sessions, movie scripts, etc. NLTK has corpora in other languages as well.

In [4]:
from nltk.corpus import gutenberg

# Printing the first 100 characters of each of the files
for fileid in gutenberg.fileids():
    print(fileid, gutenberg.raw(fileid)[:100], '...')


austen-emma.txt [Emma by Jane Austen 1816]

VOLUME I

CHAPTER I


Emma Woodhouse, handsome, clever, and rich, with a ...
austen-persuasion.txt [Persuasion by Jane Austen 1818]


Chapter 1


Sir Walter Elliot, of Kellynch Hall, in Somersetshire ...
austen-sense.txt [Sense and Sensibility by Jane Austen 1811]

CHAPTER 1


The family of Dashwood had long been settle ...
bible-kjv.txt [The King James Bible]

The Old Testament of the King James Bible

The First Book of Moses:  Called  ...
blake-poems.txt [Poems by William Blake 1789]

 
SONGS OF INNOCENCE AND OF EXPERIENCE
and THE BOOK of THEL


 SONGS  ...
bryant-stories.txt [Stories to Tell to Children by Sara Cone Bryant 1918] 


TWO LITTLE RIDDLES IN RHYME


     T ...
burgess-busterbrown.txt [The Adventures of Buster Bear by Thornton W. Burgess 1920]

I

BUSTER BEAR GOES FISHING


Bu ...
carroll-alice.txt [Alice's Adventures in Wonderland by Lewis Carroll 1865]

CHAPTER I. Down the Rabbit-Hole

Alice was ...
chesterton-ball.txt [The Ba

## Getting the data
![Fetch data](https://media.giphy.com/media/4FQMuOKR6zQRO/giphy.gif)

Data can come from a variety of sources in different formats. Natural language can be in the form of text or speech. For the purpose of this tutorial, we will be focusing on text-based processing as opposed to speech recognition and synthesis. <br>

Textual data can be stored in databases, dataframes, text files, webpages, etc. A list of text datasets can be found here: https://github.com/niderhoff/nlp-datasets. Let's create a list with a few sentences that will serve as the sample data.

In [ ]:
sample_data = ["Today is 16th July. I am in Portland, Oregon for OSCON. Currently I am " \
                "attending a Natural Language Processing tutorial."]
sample_data

You can also scrape a webpage using the `requests` and `lxml` libraries. Let's trying scraping a paragraph from the landing page for OSCON. Use 'Inspect' functionality for the webpage to get the XPath for a particular element. 

In [ ]:
from lxml import html
import requests

# Scraping data from a webpage element
page = requests.get('https://conferences.oreilly.com/oscon/oscon-or')
tree = html.fromstring(page.content)
conference_data = tree.xpath('//*[@id="reasons_row"]/div/p[1]/text()') 
print(conference_data)

## Sentence segmentation
A paragraph is nothing but a collection of sentences. Also called sentence tokenization or sentence boundary disambiguation, this process breaks up sentences by deciding where a sentence starts and ends. Challenges include recognizing ambiguous puncutation marks. For example, `.` can be used for a decimal point, an ellipsis or a period. Let's use ```sent_tokenize``` from ```nltk.tokenize``` to get sentences.

In [ ]:
from nltk.tokenize import sent_tokenize

def get_sent_tokens(data):
    """Sentence tokenization"""
    sentences = []
    for sent in data:
        sentences.extend(sent_tokenize(sent))
    print('Sentence tokens:', sentences)
    return sentences

In [ ]:
sample_sentences = get_sent_tokens(sample_data)

In [ ]:
conference_sentences = get_sent_tokens(conference_data)

## Word tokenization
A sentence is a collection of words. Word tokenization is similar to sentence tokenization, but works on words. Let's use ```word_tokenize``` from ```nltk.tokenize``` to get the words. 

In [ ]:
from nltk.tokenize import word_tokenize

def get_word_tokens(sentences):
    '''Word tokenization'''
    words = []
    for sent in sentences:
        words.extend(word_tokenize(sent))
    print('Word tokens:', words)
    return(words)

In [ ]:
sample_words = get_word_tokens(sample_sentences)

In [ ]:
conference_words = get_word_tokens(conference_sentences)

## Frequency distribution
Calculates the frequency distribution for each word in the data. Use ```nltk.probability``` from ```FreqDist``` and ```matplotlib```.

In [ ]:
import matplotlib
from nltk.probability import FreqDist
matplotlib.use('TkAgg') 

def plot_freq_dist(words, num_words = 20):
    '''Frequency distribution'''
    fdist = FreqDist(words)
    fdist.plot(num_words, cumulative=False)

In [ ]:
%matplotlib inline
plot_freq_dist(sample_words, num_words=10)

In [ ]:
plot_freq_dist(conference_words, num_words=30)

## Cleaning the data
![Cleaning data](https://media.giphy.com/media/10zsjaH4g0GgmY/giphy.gif)

Oops, we missed a crucial step! Real world data is often messy and needs to undergo cleaning. You can do a bunch of preprocessing to ensure the data is clean, like:
- Removing special characters and numbers - These are usually not important when trying to derive the semantics
- Removing stopwords - A special category of words that don't have any significance on their own and are often used as filler words or to ensure correct grammer. Eg. the, and, but, of, is, or, those, her, 
- Removing HTML tags - Raw data from webpages can often be laden with HTML tags. Use a library like `BeautifulSoup` to process and remove the tags.
- Standardizing words - This aims to consolidate different versions of the same version Eg. SMS/Twitter language, slang, misspellings 
- Converting to lower case - To ensure uniformity across all words


In [ ]:
import re
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))

def get_clean_sentences(sentences, remove_digits=False):
    '''Cleaning sentences by removing special characters and optionally digits'''
    clean_sentences = []
    for sent in sentences:
        pattern = r'[^a-zA-Z0-9\s]' if not remove_digits else r'[^a-zA-Z\s]' 
        clean_text = re.sub(pattern, '', sent)
        clean_text = clean_text.lower()
        clean_sentences.append(clean_text)
    print('Clean sentences:', clean_sentences)
    return clean_sentences

def filter_stopwords(words):
    '''Removing stopwords from given words'''
    filtered_words = [w for w in words if w not in stop_words]
    print('Filtered words:', filtered_words)
    return filtered_words

In [ ]:
sample_sentences = get_clean_sentences(sample_sentences, remove_digits = True)
sample_words = get_word_tokens(sample_sentences)
sample_words = filter_stopwords(sample_words)

In [ ]:
conference_sentences = get_clean_sentences(conference_sentences)
conference_words = get_word_tokens(conference_sentences)
conference_words = filter_stopwords(conference_words)

After cleaning the text and using tokenization, we are left with words. Words have certain properties which we'll be exploring in the next few sections. These characteristics can often be used as features for a Machine Learning model.

## POS tagging

The English language is formed of different parts of speech (POS) like nouns, verbs, pronouns, adjectives, etc. POS tagging analyzes the words in a sentences and associates it with a POS tag depending on the way it is used. Also called grammatical tagging or word-category disambiguation. Use ```nltk.pos_tag``` for the process. There are different types of tagsets used with the most common being the Penn Treebank tagset and the Universal tagset. 

![Penn POS tags](https://slideplayer.com/slide/6855236/23/images/11/Penn+TreeBank+POS+Tag+set.jpg)

In [ ]:
nltk.download('averaged_perceptron_tagger')

def get_pos_tags(words):
    '''Get the part of speech (POS) tags for the words'''
    tags=[]
    for word in words:
        tags.append(nltk.pos_tag([word]))
    print(tags)
    return tags

In [ ]:
sample_tags = get_pos_tags(sample_words)

In [ ]:
conference_tags = get_pos_tags(conference_words)

## Text processing
Text processing approaches like stemming and lemmatization help in reducing inflectional forms of words. 
### Dictionary and thesaurus
WordNet is a lexical database that also has relationships between different words. You can use synsets to find definitions, synonyms and antonyms for words. You can also find hyponyms and hypernyms using the same process. Hypernym is a generalized concept like 'programming language' whereas hyponym is a specific concept like 'Python' or 'Java'.

![Hypernym and hyponym](https://upload.wikimedia.org/wikipedia/en/thumb/1/1f/Hyponymsandhypernyms.jpg/300px-Hyponymsandhypernyms.jpg)


In [ ]:
def get_wordnet_properties(words):
    '''Returns definition, synonyms and antonyms of words'''
    for word in words:
        synonyms = []
        antonyms = []
#         hyponyms = []
#         hypernyms = []
        definitions = []
        for syn in wordnet.synsets(word):
            for lm in syn.lemmas():
                synonyms.append(lm.name())
                if lm.antonyms(): 
                    antonyms.append(lm.antonyms()[0].name())
#             hyponyms.append(syn.hyponyms())
#             hypernyms.append(syn.hypernyms())
#             definitions.append(syn.definition())
            
        print(word)
        print('Synonyms:', synonyms, '\nAntonyms:', antonyms, '\n')
#         print('Definition:', definitions, '\n')

Have you watched the series 'Friends'? Do you remember the [episode](https://youtu.be/B1tOqZUNebs?t=100) where Joey has to write a letter of recommendation for Monica and Chandler for the adoption agency? He uses a thesaurus to make himself sound smarter in the letter! Let's see if we get the same results:

'They are warm, nice people with big hearts' -> 'They are humid, prepossessing Homo Sapiens with full-sized aortic pumps'

![Joey Friends](https://media.giphy.com/media/VEsfbW0pBu145PPhOi/giphy.gif)

In [ ]:
joey_dialogue = ['they', 'are', 'warm', 'nice', 'people', 'with', 'big', 'hearts']
get_wordnet_properties(joey_dialogue)

## Word Sense Disambiguation

These synsets are also used for disambiguation, particularly Word Sense Disambiguation using Lesk Algorithm. See: http://www.nltk.org/howto/wsd.html

In [ ]:
from nltk.wsd import lesk
sent = ['I', 'went', 'to', 'the', 'bank', 'to', 'deposit', 'money', '.']
print(lesk(sent, 'bank', 'n'))

In [ ]:
sent = ['I', 'was', 'sitting', 'by', 'the', 'bank', '.']
print(lesk(sent, 'bank', 'n'))

### Stemming
Stemming tries to cut off at the ends of the words in the hope of deriving the base form. Stems aren't always real words. Use ```PorterStemmer``` from ```ntlk.stem```.

In [ ]:
from nltk.stem import PorterStemmer

def get_stems(words):
    '''Reduce the words to their base word (stem) by cutting off the ends'''
    ps = PorterStemmer()
    stems = []
    for word in words:
        stems.append(ps.stem(word))
    print(stems)
    return stems

In [ ]:
sample_stems = get_stems(sample_words)

In [ ]:
conference_stems = get_stems(conference_words)

### Lemmatization
Lemmatization groups different inflected forms of a words so they can be mapped to the same base. Lemmas are real words. More complex than stemming, context of words is also analyzed. Uses WordNet which is a lexical English database. 
Use ```WordNetLemmatizer``` from ```nltk.stem``` and provide it the POS tag along with the word. NLTK’s POS tags are in a format different from to that of wordnet lemmatizer, so a mapping is needed. https://stackoverflow.com/questions/15586721/wordnet-lemmatization-and-pos-tagging-in-python


In [ ]:
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

def get_lemma(word_tags):
    '''Reduce the words to their base word (lemma) by using a lexicon'''
    wordnet_lemmatizer = WordNetLemmatizer()
    lemma = []
    for element in word_tags:
        word = element[0][0]
        pos = element[0][1]
        tag = nltk.pos_tag([word])[0][1][0].upper()
        tag_dict = {"J": wordnet.ADJ, # Mapping NLTK POS tags to WordNet POS tags
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

        wordnet_pos = tag_dict.get(tag, wordnet.NOUN)
        lemma.append(wordnet_lemmatizer.lemmatize(word, wordnet_pos))
    print(lemma)
    return(lemma)

In [ ]:
sample_lemma = get_lemma(sample_tags)

In [6]:
conference_lemma = get_lemma(conference_tags)

NameError: name 'get_lemma' is not defined

These processes can create features that act as inputs to predictive models. It also helps in using lesser memory by making the data smaller and reducing the size of the vocabulary. Often times, these normalized words are sufficient to provide the semantics. Like in the case of understanding the meaning behind the sentences:

## Distances 
You can calculate distances between words. There are a variety of distance metrics available: https://en.wikipedia.org/wiki/String_metric. The most common ones are Levenshtein, Cosine distances and Jaccard similarity. Applications include spell checking, correction for OCRs and Machine Translation. For an implementation of a spell checker, see here: https://norvig.com/spell-correct.html

![Edit distance](https://i.stack.imgur.com/5Pjr7.png)


## Named Entity Recognition (NER) 

Also known as entity chunking or extraction, is a sub-process of information extraction. This involves identifies and classifies named entities mentions into sub-categories like person name, organization, location, time, etc.  In other words, Named Entity Recognition (NER) labels sequences of words in a text which are the names of things, such as person and company names, or gene and protein names. 

Some of the most popular NER models are here: https://towardsdatascience.com/a-review-of-named-entity-recognition-ner-using-automatic-summarization-of-resumes-5248a75de175. <br>

Example use-cases include customer support, search engine, news classification. Another emerging application is for redacting personally identifiable information (PII). A great demo of NER in action is here: https://explosion.ai/demos/displacy-ent


## Bag of words
Bag of words is an approach for text feature extraction. Just imagine a bag of popcorn, 
and each popcorn kernel represents a word that is present in the text. Each sentence can be represented as a vector
of all the words present in a vocabulary. If a word is present in the sentence, it is 1, otherwise 0.

![Bag of words](https://cdn-images-1.medium.com/max/1600/1*zMdHVQQ7HYv_mMZ5Ne-2yQ.png)

## TF-IDF
Term-frequency inverse document frequency assigns scores to words inside a document. Commonly occuring words in all documents would have less weightage.
![TF IDF](http://www.bloter.net/wp-content/uploads/2016/09/td-idf-graphic.png)

In [ ]:
import sklearn
from sklearn.feature_extraction.text import CountVectorizer

def get_bag_of_words(sentences):
    ''''''
    vectorizer = CountVectorizer()
    print(vectorizer.fit_transform(sentences).todense())
    print(vectorizer.vocabulary_) 

In [ ]:
get_bag_of_words(sample_data)

In [7]:
get_bag_of_words(conference_data)

NameError: name 'get_bag_of_words' is not defined

## Word embeddings - Word2Vec
Vector space model - represent words and sentences as vectors to get semantic relationships. A really good tutorial for Word2Vec is here: https://www.kaggle.com/alvations/word2vec-embedding-using-gensim-and-nltk

![Word2Vec](http://www.flyml.net/wp-content/uploads/2016/11/w2v-3-samples.png)

## Machine Learning using Natural Language Processing
Machine Learning includes two approaches: supervised and unsupervised. Supervised learning works on data that already has labels i.e. they provide supervision to the model. Eg. Classification, Regression. Unsupervised learning is to find out the inherent structure present in the data and there are no labels i.e. no supervision. Eg. Clustering.

A lot of these text properties can be used as features for Machine Learning systems. One specific case is text classification. A more detailed resource is here: https://www.nltk.org/book/ch06.html

## Sentiment analysis

NLTK's VADER algorithm is used to detect polarity of words and establish the overall sentiment (compound score) for sentences. We're using a small sample of IMDB review titles for Captain Marvel. 

In [ ]:
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer 

def get_sentiment(data):
    '''Get sentiment of sentences using VADER algorithm'''
    scorer = SentimentIntensityAnalyzer()
    for sentence in reviews:
        print(sentence)
        ss = scorer.polarity_scores(sentence)
        for k in ss:
            print('{0}: {1}, ' .format(k, ss[k]), end='')
        print()  

In [ ]:
reviews = ["I've watched this movie for the record, cause I had no particular expectations for it.",
          "It's fine.",
          "Not bad, not great either.",
          "Captain Marvel is another fine Marvel adventure.",
          "Captivating story and great visuals.",
          "Confusing boring childish.",
          "Strong Hero. Weak Film."]

get_sentiment(reviews)

## Topic modeling
Topic modeling is an unsupervised ML method used to find inherent structure in documents. It learns
representations of topics in documents which allows grouping of different documents together. We will
use ```Gensim``` library and Latent Dirichlet Allocation (LDA) for this.

LDA’s approach to topic modeling is it considers each document as a collection of topics in a certain proportion. And each topic as a collection of keywords, again, in a certain proportion.

Once you provide the algorithm with the number of topics, all it does it to rearrange the topics distribution within the documents and keywords distribution within the topics to obtain a good composition of topic-keywords distribution.

Adapted from https://kleiber.me/blog/2017/07/22/tutorial-lda-wikipedia/

![Topic modeling](https://i.stack.imgur.com/vI8Lc.png)

In [ ]:
import wikipedia, random

def fetch_data(article_names):
    '''Fetching the data from given Wikipedia articles'''
    wikipedia_random_articles = wikipedia.random(2)
    wikipedia_random_articles.extend(article_names)
    wikipedia_random_articles
    print(wikipedia_random_articles)
    
    wikipedia_articles = []
    for wikipedia_article in wikipedia_random_articles:
        wikipedia_articles.append([wikipedia_article, 
                                   wikipedia.page(wikipedia_article).content])
    return wikipedia_articles

In [ ]:
nltk.download('stopwords')    
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer

def clean(article):
    '''Cleaning the article contents and getting the word stems'''
    title, document = article
    tokens = RegexpTokenizer(r'\w+').tokenize(document.lower())
    tokens_clean = [token for token in tokens if token not in 
                    stopwords.words('english')]
    tokens_stemmed = [PorterStemmer().stem(token) for token 
                      in tokens_clean]
    return (title, tokens_stemmed)

In [ ]:
from gensim import corpora, models
import gensim

article_names = ['Music', 'Grace Hopper', 'Omaha', 'Data', 'Compiler']
wikipedia_articles = fetch_data(article_names)
wikipedia_articles
wikipedia_articles_clean = list(map(clean, wikipedia_articles))

In [ ]:
article_contents = [article[1] for article in wikipedia_articles_clean]
dictionary = corpora.Dictionary(article_contents)
corpus = [dictionary.doc2bow(article) for article in 
          article_contents[:-1]] # All except 'Compiler'

In [ ]:
lda_model = gensim.models.ldamodel.LdaModel(corpus, num_topics=6, 
                                            id2word = dictionary, 
                                            passes=100)

topic_results = lda_model.print_topics(num_topics=6, num_words=5)
topic_results

In [ ]:
from matplotlib import pyplot as plt
from wordcloud import WordCloud, STOPWORDS
import matplotlib.colors as mcolors

cols = [color for name, color in mcolors.TABLEAU_COLORS.items()]  # more colors: 'mcolors.XKCD_COLORS'

cloud = WordCloud(background_color='white',
                  width=2500,
                  height=1800,
                  max_words=10,
                  colormap='tab10',
                  color_func=lambda *args, **kwargs: cols[i],
                  prefer_horizontal=1.0)

topics = lda_model.show_topics(formatted=False)

fig, axes = plt.subplots(3, 2, figsize=(10,10), sharex=True, sharey=True)

for i, ax in enumerate(axes.flatten()):
    fig.add_subplot(ax)
    topic_words = dict(topics[i][1])
    cloud.generate_from_frequencies(topic_words, max_font_size=300)
    plt.gca().imshow(cloud)
    plt.gca().set_title('Topic ' + str(i), fontdict=dict(size=16))
    plt.gca().axis('off')


plt.subplots_adjust(wspace=0, hspace=0)
plt.axis('off')
plt.margins(x=0, y=0)
plt.tight_layout()
plt.show()

In [ ]:
print(list(lda_model[[dictionary.doc2bow(article_contents[-1])]]))

## Resources

- More on NLP https://monkeylearn.com/blog/definitive-guide-natural-language-processing/
- A very comprehensive list of resources by Penn https://www.seas.upenn.edu/~romap/nlp-resources.html
- Peter Norvig's spell corrector http://norvig.com/spell-correct.html
- Applications and datasets https://machinelearningmastery.com/datasets-natural-language-processing/
- More datasets https://gengo.ai/datasets/the-best-25-datasets-for-natural-language-processing/
- https://towardsdatascience.com/text-analytics-topic-modelling-on-music-genres-song-lyrics-deb82c86caa2
- Collection of tutorials https://medium.com/machine-learning-in-practice/over-200-of-the-best-machine-learning-nlp-and-python-tutorials-2018-edition-dd8cf53cb7dc
- Text classification https://textminingonline.com/dive-into-nltk-part-vii-a-preliminary-study-on-text-classification


## Contact

You can contact me at:

- Personal website: https://gjena.github.io/
- LinkedIn: https://www.linkedin.com/in/grishmajena/
- Twitter: @DebateLover